In [19]:
import numpy as np
import pandas as pd
import mrcfile
import torch
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from scipy.spatial.transform import Rotation

import napari
import torch_fourier_rescale

## Load in volume and downscale

A pre-simulated volume is loaded into memory, and then downscale it to a desired pixel size (for better calculation efficiency)

In [ ]:
with mrcfile.open("/Users/mgiammar/Downloads/lsu.mrc", mode="r") as f:
    volume = f.data.copy()
    original_pixel_size = f.voxel_size
    original_pixel_size = original_pixel_size["x"].item()  # assuming isotropic

print(f"Original pixel size: {original_pixel_size:.3f}")
print(f"Volume shape: {volume.shape}")

# Use the torch-fourier-rescale package to rescale the volume
desired_pixel_size = 8.0
volume_downsampled, new_spacing = torch_fourier_rescale.fourier_rescale_3d(
    image=torch.from_numpy(volume),
    source_spacing=original_pixel_size,
    target_spacing=desired_pixel_size,
)

new_pixel_size = new_spacing[0].item()  # Assuming isotropic spacing
print()
print(f"New pixel size: {new_pixel_size:.3f}")
print(f"Downsampled volume shape: {volume_downsampled.shape}")

Original pixel size: 0.950
Volume shape: (512, 512, 512)

New pixel size: 8.107
Downsampled volume shape: torch.Size([60, 60, 60])


### View the volume using Napari

Doing an initial visualization of the structure in Napari without any translations or rotations

In [4]:
viewer = napari.Viewer()
viewer.add_image(
    volume_downsampled.numpy(),
    name="Downsampled Volume",
    scale=[new_pixel_size] * 3,  # Assuming isotropic spacing
)
napari.run()

2025-05-29 15:51:50.859 python[2971:130517848] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-29 15:51:50.859 python[2971:130517848] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Creating slab with translations and rotations

Here, the slab is created to be the same size as the original micrograph if it were down-sampled.
This means the slab can be directly overlaid on the micrograph to visualize structure position

In [ ]:
def create_empty_slab(
    image_shape: tuple[int, int],
    image_pixel_size: float,  # Angstroms
    slab_pixel_size: float,  # Angstroms
    slab_thickness: float,  # Angstroms
) -> np.ndarray:
    """Create an empty slab with the specified pixel size and thickness.

    Parameters
    ----------
    image_shape : tuple[int, int]
        Shape of the image (height, width).
    image_pixel_size : float
        Pixel size of the image, in Angstroms.
    slab_pixel_size : float
        Desired voxel size of the slab, in Angstroms (isotropic).
    slab_thickness : float
        Thickness of the slab, in Angstroms.
    """
    # Slab spans the same physical area as the image, but will have a different
    # voxel pitch.
    height = image_shape[0] * image_pixel_size
    width = image_shape[1] * image_pixel_size

    slab_height = int(height / slab_pixel_size)
    slab_width = int(width / slab_pixel_size)
    slab_depth = int(slab_thickness / slab_pixel_size)

    slab = np.zeros((slab_height, slab_width, slab_depth), dtype=np.float32)

    return slab

### Helper function to place a smaller volume into a larger volume

In [ ]:
def place_into_larger_volume(
    small_volume: np.ndarray,
    large_volume: np.ndarray,
    position: tuple[int, int, int],
    rot_x: float = 0.0,
    rot_y: float = 0.0,
    rot_z: float = 0.0,
) -> np.ndarray:
    """Transform a small volume and place it into a larger volume.

    NOTE: The rotations are applied in xyz format. Other conventions will need
    to adjust accordingly.

    Parameters
    ----------
    small_volume : np.ndarray
        The small volume to be placed, shape (depth, height, width).
    large_volume : np.ndarray
        The larger volume into which the small volume will be placed, shape
        (depth, height, width).
    position : tuple[int, int, int]
        The (x, y, z) position in the larger volume where the small volume will be placed.
    rot_x : float, optional
        Rotation angle around the x-axis in degrees, default is 0.0.
    rot_y : float, optional
        Rotation angle around the y-axis in degrees, default is 0.0.
    rot_z : float, optional
    """
    # Sequentially rotate the small volume
    volume_rotated = rotate(small_volume, rot_x, axes=(1, 2), reshape=False)
    volume_rotated = rotate(volume_rotated, rot_y, axes=(0, 2), reshape=False)
    volume_rotated = rotate(volume_rotated, rot_z, axes=(0, 1), reshape=False)

    # Calculate the position in the larger volume (only integer coordinates)
    x, y, z = position
    x_end = x + volume_rotated.shape[0]
    y_end = y + volume_rotated.shape[1]
    z_end = z + volume_rotated.shape[2]

    # Check if the small volume fits into the larger volume
    if (
        x < 0
        or y < 0
        or z < 0
        or x_end > large_volume.shape[0]
        or y_end > large_volume.shape[1]
        or z_end > large_volume.shape[2]
    ):
        print(f"Position: {position}")
        print(f"Small volume shape: {small_volume.shape}")
        print(f"Larger volume shape: {large_volume.shape}")
        raise ValueError("Volume is out of bounds!")

    # Place the rotated small volume into the larger volume
    large_volume[x:x_end, y:y_end, z:z_end] += volume_rotated

    return large_volume

In [14]:
slab = create_empty_slab(
    image_shape=(4096, 4096),
    image_pixel_size=0.936,
    slab_pixel_size=new_pixel_size,
    slab_thickness=2400.0,
)

slab.shape

(472, 472, 296)

In [ ]:
x = np.linspace(100, 400, 4)
y = np.linspace(100, 400, 4)

for i in range(x.size):
    slab = place_into_larger_volume(
        small_volume=volume_downsampled.numpy(),
        large_volume=slab,
        position=(int(x[i]), int(y[i]), 100),  # Place in the first slice
        rot_x=0.0,
        rot_y=0.0,
        rot_z=0.0,
    )

In [16]:
# Render the slab in napari
viewer = napari.Viewer()
viewer.add_image(
    slab,
    name="Slab with Downsampled Volume",
    scale=[new_pixel_size] * 3,  # Assuming isotropic spacing
)
napari.run()

## Helper function to render volume from results DataFrame

In [25]:
def construct_slab_from_results(
    volume: np.ndarray,
    results: pd.DataFrame,
    image_shape: tuple[int, int],
    image_pixel_size: float,
    slab_pixel_size: float,
    slab_thickness: float,
):
    """Takes in a results DataFrame and constructs a slab for the volume."""
    slab = create_empty_slab(
        image_shape=image_shape,
        image_pixel_size=image_pixel_size,
        slab_pixel_size=slab_pixel_size,
        slab_thickness=slab_thickness,
    )

    for i, row in results.iterrows():
        print(f"Placing volume {i + 1} of {len(results)}")
        # Get full image position and convert to slab coordinates
        position = np.ndarray(
            [
                int(row["pos_x_img_angstrom"]),
                int(row["pos_y_img_angstrom"]),
                int(row["relative_defocus"] + slab_thickness / 2),
            ]
        )
        position = np.round(position / slab_pixel_size).astype(int)
        position = tuple(position)
        print(f"Position in slab: {position}")

        orientation = [row["phi"], row["theta"], row["psi"]]

        # Convert 'ZYZ' Euler angles into 'xyz' rotation angles
        rot = Rotation.from_euler("ZYZ", orientation, degrees=True)
        rot_x, rot_y, rot_z = rot.as_euler("xyz", degrees=True)

        slab = place_into_larger_volume(
            small_volume=volume,
            large_volume=slab,
            position=position,
            rot_x=rot_x,
            rot_y=rot_y,
            rot_z=rot_z,
        )

    return slab


df = pd.read_csv("/Users/mgiammar/Downloads/large_image_test_results.csv")

slab = construct_slab_from_results(
    volume=volume_downsampled.numpy(),
    results=df,  # Assuming df is a DataFrame with the required columns
    image_shape=(4096, 4096),  # Shape of the original image
    image_pixel_size=original_pixel_size,  # Original pixel size in Angstroms
    slab_pixel_size=new_pixel_size,  # New pixel size in Angstroms
    slab_thickness=2400.0,  # Thickness of the slab in Angstroms
)

Placing volume 1 of 279
Position in slab: (array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(573, 1800)), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(573, 1800)), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(573, 1800)), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(573, 1800)), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 

ValueError: too many values to unpack (expected 3)